## Objetivo: Avaliar a qualidade da base e tratar/processar informações

### Importações

In [15]:
import pandas as pd
import re
import nltk
from unidecode import unidecode
from dateutil import parser

### Carregando base e avaliando qualidade

In [16]:
df = pd.read_csv("base_fake_news.csv")
df.head()

,ID,Noticia,Categoria,Data,Fake
0,1,Kátia Abreu diz que vai colocar sua expulsão e...,politica,2017-11-30,1
1,10,"Dr. Ray peita Bolsonaro, chama-o de conservad...",politica,2017-11-24,1
2,100,Reinaldo Azevedo desmascarado pela Polícia Fed...,politica,2017-05-23,1
3,1000,Relatório assustador do BNDES mostra dinheiro ...,politica,24/07/2017,1
4,1001,"Radialista americano fala sobre o PT: ""Eles ve...",politica,25/07/2017,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         7200 non-null   int64 
 1   Noticia    7200 non-null   object
 2   Categoria  7200 non-null   object
 3   Data       7200 non-null   object
 4   Fake       7200 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 281.4+ KB


### Avaliando dados e aplicando formatação, tratamento e processamento

In [18]:
# Avaliando quantos valores únicos possuem cada variável

df.agg(["nunique", "unique"])

,ID,Noticia,Categoria,Data,Fake
nunique,3600,7199,6,3689,2
unique,"[1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...",[Kátia Abreu diz que vai colocar sua expulsão ...,"[politica, tv_celebridades, sociedade_cotidian...","[2017-11-30, 2017-11-24, 2017-05-23, 24/07/201...","[1, 0]"


In [19]:
# Avaliando se existe padronização nos formatos de data através do tamanho da string

len_data = df["Data"].str.len()
vc = len_data.value_counts()
vc

10    3742
16    2162
19     405
22     277
21     179
23     121
8      100
9       90
20      69
18      54
7        1
Name: Data, dtype: int64

In [20]:
# Avaliando cinco exemplos de datas com base nos tamanhos das strings

for nc in vc.index:
  print(df.loc[df["Data"].str.len()==nc, "Data"].values[:5])

['2017-11-30' '2017-11-24' '2017-05-23' '24/07/2017' '25/07/2017']
['31/10/2017 10h17' '05/04/2017 09h08' '22/12/2017 11h57'
 '28/07/2017 13h40' '06/01/2018 09h29']
['03 de março de 2015' '16 de julho de 2016' '01 de julho de 2017'
 '24 de março de 2018' '08 de março de 2018']
['16 de novembro de 2017' '21 de dezembro de 2017'
 '18 de dezembro de 2017' '17 de novembro de 2014'
 '10 de dezembro de 2016']
['20 de janeiro de 2015' '16 de outubro de 2017' '14 de janeiro de 2018'
 '08 de janeiro de 2018' '24 de janeiro de 2018']
['20 de fevereiro de 2018' '22 de fevereiro de 2018'
 '23 de fevereiro de 2018' '20 de fevereiro de 2016'
 '06 de fevereiro de 2018']
['3/1/2018' '4/1/2018' '4/1/2018' '4/1/2018' '4/1/2018']
['12/1/2018' '7/12/2017' '1/12/2017' '14/7/2017' '25/7/2016']
['21 de agosto de 2017' '01 de agosto de 2017' '21 de agosto de 2016'
 '04 de agosto de 2017' '22 de agosto de 2017']
['25 de maio de 2017' '03 de maio de 2017' '13 de maio de 2016'
 '18 de maio de 2017' '03 de maio d

In [21]:
# Dicionário para pré-processamento das datas

dict_month = {
    "janeiro": "01",
    "fevereiro": "02",
    "março": "03",
    "marco": "03", # Possibilidade de existir março com ou sem cedilha
    "abril": "04",
    "maio": "05",
    "junho": "06",
    "julho": "07",
    "agosto": "08",
    "setembro": "09",
    "outubro": "10",
    "novembro": "11",
    "dezembro": "12"
}

In [22]:
# Função para processamento das datas

def date_preprocessing(date):

  date = date.lower()

  for k, v in dict_month.items():
    date = re.sub(" de "+k+" de ", "/"+v+"/", date)

  return parser.parse(date)

In [23]:
# Aplicando o pré processamento nas datas

df["data_proc"] = df["Data"].apply(date_preprocessing)
df["data_proc"]

0       2017-11-30 00:00:00
1       2017-11-24 00:00:00
2       2017-05-23 00:00:00
3       2017-07-24 00:00:00
4       2017-07-25 00:00:00
               ...         
7195    2016-11-22 00:00:00
7196    2018-09-02 08:25:00
7197    2016-12-31 05:00:00
7198    2017-09-15 20:46:00
7199    2018-08-01 11:58:00
Name: data_proc, Length: 7200, dtype: object

In [24]:
# Contagem dos anos obtidos

Ano = pd.Series([dt.year for dt in df["data_proc"].values])
Ano.value_counts()

2017    3711
2016    1864
2018    1308
2015     275
2012      10
2014      10
2010       6
2011       5
2013       5
201        3
2009       3
dtype: int64

In [25]:
# Avaliando anos com erro e atribuindo as datas de seus respectivos pares

sub_df = df.loc[Ano==201]
columns = ["Data", "data_proc"]

for id in sub_df["ID"].unique():

  df.loc[(df["ID"]==id)&(df.index.isin(sub_df.index)), columns] = df.loc[(df["ID"]==id)&(~df.index.isin(sub_df.index)), columns].values

df.loc[df["ID"].isin(sub_df["ID"].values)]

,ID,Noticia,Categoria,Data,Fake,data_proc
13,101,URGENTE: Janot pede prisão de Aécio Neves. O p...,politica,2017-05-22,1,2017-05-22 00:00:00
700,163,"Polícia Federal acaba de indiciar Lula, a espo...",politica,2016-12-12,1,2016-12-12 00:00:00
2891,3601,"Há 400 anos, Nostradamus previu a vitória de ...",tv_celebridades,24 de julho de 2017,1,2017-07-24 00:00:00
3613,101,CPI aprova relatório final e recomenda investi...,politica,2017-05-22,0,2017-05-22 00:00:00
4300,163,"Antes do Natal, Lula tentou tirar, novamente, ...",politica,2016-12-12,0,2016-12-12 00:00:00
6491,3601,"As lições de educação moral e cívica, que o ba...",tv_celebridades,24 de julho de 2017,0,2017-07-24 00:00:00


In [26]:
# Transformando no formato datetime

df["data_proc"] = pd.to_datetime(df["data_proc"])
df["data_proc"]

0      2017-11-30 00:00:00
1      2017-11-24 00:00:00
2      2017-05-23 00:00:00
3      2017-07-24 00:00:00
4      2017-07-25 00:00:00
               ...        
7195   2016-11-22 00:00:00
7196   2018-09-02 08:25:00
7197   2016-12-31 05:00:00
7198   2017-09-15 20:46:00
7199   2018-08-01 11:58:00
Name: data_proc, Length: 7200, dtype: datetime64[ns]

In [27]:
'''
Tratamento dos textos - Vai retornar o texto de entrada em três estados distintos:
   - Texto limpo: Texto sem problemas de formatação provenientes de diferentes codificações
   - Texto tratado: Texto sem acentos e pontuações gráficas, além da remoção de stopwords
   - Texto stemizado: Texto tratado com tokens stemizados
'''

stopwords = ['a', 'à', 'adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'anos', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'área', 'as', 'às', 'assim', 'até', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'custa', 'da', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'enquanto', 'entre', 'era', 'eram', 'éramos', 'és', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estás', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou', 'etc', 'eu', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes', 'grupo', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'la', 'lá', 'lado', 'lhe', 'lhes', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muita', 'muitas', 'muito', 'muitos', 'na', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'nem', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nível', 'no', 'noite', 'nome', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'número', 'nunca', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'paucas', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'qual', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'são', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sejamos', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'sétima', 'sétimo', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'te', 'tem', 'tém', 'têm', 'temos', 'tendes', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo', 'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'você', 'vocês', 'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero', '_']
stemmer = nltk.stem.RSLPStemmer()

def txt_preprocessing(txt):
  
  # Corrigindo alguns problemas de decodificação
  txt1 = re.sub("\x91|\x92|\x93|\x94", '"', txt)
  txt1 = re.sub("\x96", "-", txt1)

  txt2 = txt1.lower()                       # Passando o texto para minúsculo
  txt2 = re.sub(r"\d", " ", txt2)           # Removendo digitos
  txt2 = re.sub(r"[^\w\s]", " ", txt2)      # Removendo qualquer coisa que não seja letra ou espaço
  txt2 = re.sub(r"\b\w{1,2}\b", " ", txt2)  # Removendo palavras com apenas uma ou duas letras
  txt2 = re.sub(r"\s+", " ", txt2)          # Removendo espaços em excesso por espaço único

  txt2 = " ".join([unidecode(tkn) for tkn in txt2.split() if tkn not in stopwords]) # Removendo stopwords e acentos gráficos

  txt3 = " ".join([stemmer.stem(tkn) for tkn in txt2.split()]) # Stemizando

  return txt1, txt2, txt3 # Texto sem problemas de decodificação, texto limpo não stemizado, texto limpo stemizado


df[["txt_limpo", "txt_tratado", "txt_stemizado"]] = df.apply(lambda f: txt_preprocessing(f.Noticia), axis=1, result_type="expand")
df.head()

,ID,Noticia,Categoria,Data,Fake,data_proc,txt_limpo,txt_tratado,txt_stemizado
0,1,Kátia Abreu diz que vai colocar sua expulsão e...,politica,2017-11-30,1,2017-11-30,Kátia Abreu diz que vai colocar sua expulsão e...,katia abreu colocar expulsao moldura reclamar ...,kat abr coloc expulsa mold reclam sen kat abr ...
1,10,"Dr. Ray peita Bolsonaro, chama-o de conservad...",politica,2017-11-24,1,2017-11-24,"Dr. Ray peita Bolsonaro, chama-o de ""conservad...",ray peita bolsonaro chama conservador fake ent...,ray peit bolsonar cham conserv fak entrev dani...
2,100,Reinaldo Azevedo desmascarado pela Polícia Fed...,politica,2017-05-23,1,2017-05-23,Reinaldo Azevedo desmascarado pela Polícia Fed...,reinaldo azevedo desmascarado policia federal ...,reinald azeved desmascar polic feder ferrenh c...
3,1000,Relatório assustador do BNDES mostra dinheiro ...,politica,24/07/2017,1,2017-07-24,Relatório assustador do BNDES mostra dinheiro ...,relatorio assustador bndes mostra dinheiro pub...,relatori assust bnd mostr dinh publ brasil jor...
4,1001,"Radialista americano fala sobre o PT: ""Eles ve...",politica,25/07/2017,1,2017-07-25,"Radialista americano fala sobre o PT: ""Eles ve...",radialista americano fala vendem ilusao povo b...,radial americ fal vend ilusa pov brasil pens d...


In [28]:
# Salvando base final com dados tratados

df[["ID", "txt_limpo", "txt_tratado", "txt_stemizado", "Categoria", "data_proc", "Fake"]].to_csv("base_fake_news_trat.csv", index=False)